In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, datasets,models

import io
import imageio
import pathlib
from IPython.display import Image, display
from ipywidgets import widgets, Layout, HBox
from PIL import Image

from tensorflow.keras.preprocessing.image import load_img, img_to_array
import time

2022-08-27 15:48:58.036814: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64::/home/zhenzey/src/openmpi/lib:/usr/local/lib
2022-08-27 15:48:58.036831: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
# Loading video transformer model
ckpt_vivit = "ckpts/vivit"
model_vivit = tf.keras.models.load_model(ckpt_vivit)

# Loading CNN model
ckpt_cnn = "ckpts/cnn"
model_cnn = tf.keras.models.load_model(ckpt_cnn)

2022-08-25 10:15:52.469188: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64:/usr/local/cuda/lib64:/usr/local/cuda-10.2/ib64:/usr/local/cuda-10.0/lib64/::/home/zhenzey/src/openmpi/lib:/usr/local/lib
2022-08-25 10:15:52.469306: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64:/usr/local/cuda/lib64:/usr/local/cuda-10.2/ib64:/usr/local/cuda-10.0/lib64/::/home/zhenzey/src/openmpi/lib:/usr/local/lib
2022-08-25 10:15:52.469358: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No suc

In [7]:
# Load dataset
dataset = np.load("./datasets/S11.npy")
indexes = np.arange(dataset.shape[0])
#np.random.shuffle(indexes)
test_index = indexes[int(0.9 * dataset.shape[0]) : ]
test_dataset = dataset[test_index]

# Normalize the data to the 0-1 range.
test_dataset = test_dataset / 255

# We'll define a helper function to shift the frames, where
# `x` is frames 0 to n - 1, and `y` is frames 1 to n.
def create_shifted_frames(data):
    x = data[:, 0 : 8, :, :]
#     for i in range(14):

#     	y = np.concatenate((y, data[:, i + 2 : i + 3, :, :]), axis=-1)
    return x


# Apply the processing function to the datasets.
x_test = create_shifted_frames(test_dataset)

y_test = np.load("./datasets/labels_test.npy")[200:]

# Inspect the dataset.

print("Test Dataset Shapes: " + str(x_test.shape) + ", " + str(y_test.shape))

Test Dataset Shapes: (200, 8, 32, 32, 3), (200, 64)


In [17]:
def vivit_cnn(index):
    x, y = x_test[index], y_test[index]
    pred = model_vivit.predict(np.expand_dims(x, axis=0))
    pred =  np.squeeze(pred, axis=0)
    complete_frames = np.concatenate((x, pred), axis=0)
    seq = (model_cnn.predict(np.expand_dims(complete_frames, axis=0)) > 0.5).astype("int")
    
    error = np.sum((np.squeeze(seq)-y) ** 2)
    return seq, error

def create_3D_geo(partition, material_dis_pred, material_dis_gt, index):
    """
    partition = (nx, ny, nz)
    """
    # Test order
    nx, ny, nz = partition
    data_pred = np.zeros((nz, ny, nx * 2))
    data_gt = np.zeros((nz, ny, nx * 2))
    fill = np.ones((nz, ny, nx * 2))
    for i in range(nx, nx * 2):
        fill[:, :, i] = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                order = int(i * ny * nz + j * nz + k)
                data_pred[k][j][i] = material_dis_pred[order]
                data_gt[k][j][i] = material_dis_gt[order]
    
    facecolors_pred = np.where(data_pred, "white", "red")
    facecolors_gt = np.where(data_gt, "white", "red")
    ax = plt.figure(figsize=(8, 8)).add_subplot(projection='3d')
    ax.voxels(fill, facecolors=facecolors_pred,  edgecolors = "black")
    plt.axis("off")
    
    plt.savefig("./prediction/geos/" + str(index) + "_pred.png", dpi=300)
    plt.close()

    ax = plt.figure(figsize=(8, 8)).add_subplot(projection='3d')
    ax.voxels(fill, facecolors=facecolors_gt,  edgecolors = "black")
    plt.axis("off")
    
    plt.savefig("./prediction/geos/" + str(index) + "_gt.png", dpi=300)
    plt.close()
    
    

def predict_plot(index):

    x, y = x_test[index], y_test[index]
    # Pick the first/last ten frames from the example.


    frames = x
    original_frames = []

    for i in range(1):
        for j in range(8):
            original_frames.append(y[j, :, :, i * 3 : i * 3 + 3].tolist())
    original_frames = np.array(original_frames)

    # Predict a new set of 18 frames.
    for _ in range(1):
        # Extract the model's prediction and post-process it.
        new_prediction = model_trained.predict(np.expand_dims(frames, axis=0))
        new_prediction = np.squeeze(new_prediction, axis=0)
        print(new_prediction.shape)
        #predicted_frame = np.expand_dims(new_prediction[-1, ...], axis=0)

        # Extend the set of prediction frames.
        #frames = np.concatenate((frames, predicted_frame), axis=0)
    new_frames = []
    for i in range(1):
        for j in range(8):
            new_frames.append(new_prediction[j, :, :, i * 3 : i * 3 + 3].tolist())
    new_frames = np.array(new_frames)
    fig, axs = plt.subplots(2, 16, figsize=(16, 3))

    for i in range(8):
        axs[0, i].imshow(np.squeeze(x)[i])
        axs[0, i].axis("off")
    for i in range(8):
        axs[0, i + 8].imshow(np.squeeze(original_frames[i]))
        axs[0, i + 8].axis("off")
    for i in range(16):
        if i < 8:
            axs[1, i].axis("off")
        else:
            axs[1, i].imshow(np.squeeze(new_frames[i - 8]))
            axs[1, i].axis("off")

    plt.savefig("./prediction/field_maps/" + str(index + 1) + ".png", dpi=300)
    plt.close()
            


In [ ]:
path_field_map = "./predictions/field_maps/"
# Check whether the specified path exists or not
isExist = os.path.exists(path_field_map)
if not isExist:

   # Create a new directory because it does not exist
    os.makedirs(path_field_map)

path_geo = "./predictions/geos/"

# Check whether the specified path exists or not
isExist = os.path.exists(path_geo)
if not isExist:

   # Create a new directory because it does not exist
    os.makedirs(path_geo)
    
errors = []
for index in range(200):
    seq, error = vivit_cnn(index)
    predict_plot(index)
    errors.append(error)
    create_3D_geo((2, 4, 4), np.squeeze(seq), np.squeeze(y_test[index]), index)
np.save("geo_diff.npy", np.array(errors))